In [ ]:
%load_ext autoreload
%autoreload 2

# Split input pairs into train and test sets

In [ ]:
from src.data.familysearch import train_test_split, load_train_test

In [ ]:
in_path = "s3://nama-data/data/interim/tree-hr-surname-similar.csv.gz"
train_path = "s3://nama-data/data/processed/tree-hr-surname-similar-train.csv.gz"
test_path = "s3://nama-data/data/processed/tree-hr-surname-similar-test.csv.gz"

In [ ]:
train_test_split(in_path, train_path, test_path, starting_test_size=0.55)

In [ ]:
train, test = load_train_test(train_path, test_path)

input_names_train, weighted_actual_names_train, candidate_names_train = train
input_names_test, weighted_actual_names_test, candidate_names_test = test

In [ ]:
print("train input, weighted actual, candidate")
print(len(input_names_train))
print(len(weighted_actual_names_train))
print(len(candidate_names_train))

In [ ]:
print("test input, weighted actual, candidate")
print(len(input_names_test))
print(len(weighted_actual_names_test))
print(len(candidate_names_test))